# BoxBunny Component Testing Notebook
This notebook provides individual testing cells for each component.
Settings tuned here will reflect in the ROS nodes.

**Structure:**
1. Environment Setup
2. Camera & Action Recognition Model
3. IMU Calibration & Punch Detection
4. LLM Tuning
5. Drill Configuration
6. Full System Integration Tests


## 1. Environment Setup
Run this first to set up paths and environment.


In [ ]:
%%bash
set -e
cat > /tmp/boxbunny_env.sh <<'SH'
source ~/miniconda3/etc/profile.d/conda.sh
conda activate boxing_ai
export PYTHONNOUSERSITE=1
export SETUPTOOLS_USE_DISTUTILS=stdlib
export QT_QPA_PLATFORM_PLUGIN_PATH="$CONDA_PREFIX/plugins/platforms"
export QT_PLUGIN_PATH="$CONDA_PREFIX/plugins"
export LD_LIBRARY_PATH="$CONDA_PREFIX/lib:$LD_LIBRARY_PATH"
source /opt/ros/humble/setup.bash
source /home/boxbunny/Desktop/doomsday_integration/boxing_robot_ws/install/setup.bash
SH
echo "Environment file created at /tmp/boxbunny_env.sh"


In [ ]:
# Quick build (run after code changes)
!bash -lc 'source /tmp/boxbunny_env.sh && cd /home/boxbunny/Desktop/doomsday_integration/boxing_robot_ws && colcon build --symlink-install'


---
## 1.5 GUI Preview (Standalone)
Test the GUI layout and UI without running any other ROS nodes.
This is useful for UI development and styling changes.


In [ ]:
# Run GUI standalone (no backend required)
# Useful for testing UI layout and making style changes
!bash -lc 'source /tmp/boxbunny_env.sh && ros2 run boxbunny_gui boxbunny_gui'


In [ ]:
# Alternative: Direct Python execution for faster iteration
# This runs the GUI directly without rebuilding
!bash -lc 'source /tmp/boxbunny_env.sh && cd /home/boxbunny/Desktop/doomsday_integration/boxing_robot_ws/src/boxbunny_gui/boxbunny_gui && python gui_main.py'


> **Tip:** After modifying , just re-run the cell above to see changes immediately.
> The GUI will show with empty data but all UI elements are visible for styling.


---
## 2. Camera & Action Recognition Model
Test the RealSense D435i camera and action recognition model performance.


### 2.1 Camera Test (RealSense D435i)


In [ ]:
# Direct camera test (press q to quit)
import pyrealsense2 as rs
import cv2
import numpy as np

pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
pipeline.start(config)
align = rs.align(rs.stream.color)

try:
    for _ in range(300):
        frames = pipeline.wait_for_frames()
        aligned = align.process(frames)
        color_frame = aligned.get_color_frame()
        depth_frame = aligned.get_depth_frame()
        if not color_frame or not depth_frame:
            continue
        rgb = np.asanyarray(color_frame.get_data())
        depth = np.asanyarray(depth_frame.get_data())
        depth_vis = cv2.applyColorMap(cv2.convertScaleAbs(depth, alpha=0.03), cv2.COLORMAP_JET)
        combined = np.hstack([rgb, depth_vis])
        cv2.imshow('D435i RGB + Depth', combined)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
finally:
    pipeline.stop()
    cv2.destroyAllWindows()


### 2.2 Action Recognition Model Test
Test the RGBD action recognition model directly.
Tune parameters here and apply to ROS nodes.


In [ ]:
# Run the live inference GUI for action recognition
# Adjust --fps, --rgb-res, --depth-res to tune performance
!bash -lc 'source /tmp/boxbunny_env.sh && \
  cd /home/boxbunny/Desktop/doomsday_integration/boxing_robot_ws/action_prediction && \
  python tools/inference/live_infer_rgbd.py \
    --model-config configs/rgbd_boxing_anticipation.py \
    --model-checkpoint /home/boxbunny/Desktop/doomsday_integration/best_acc_82.4_epoch_161.pth \
    --yolo-model checkpoints/yolo26m.pt \
    --fps 30 \
    --rgb-res 640x480 \
    --depth-res 640x480'


---
## 3. IMU Calibration & Punch Detection
Calibrate the IMU punch classifier for accurate detection.


### 3.1 IMU Raw Data Test


In [ ]:
# Check IMU raw data is being published
!bash -lc 'source /tmp/boxbunny_env.sh && ros2 topic echo -n 5 /imu/data'


### 3.2 IMU Punch Calibration
Use this to calibrate punch detection thresholds.


In [ ]:
# Launch IMU punch GUI for calibration
# This provides real-time visualization and calibration controls
!bash -lc 'source /tmp/boxbunny_env.sh && ros2 run boxbunny_imu imu_punch_gui'


### 3.3 Calibrate Individual Punch Types
Service calls to calibrate specific punch types.


In [ ]:
# Calibrate jab/cross (throw jabs for 2.5 seconds after running)
!bash -lc 'source /tmp/boxbunny_env.sh && ros2 service call /calibrate_imu_punch boxbunny_msgs/srv/CalibrateImuPunch "{punch_type: jab_or_cross, duration_s: 2.5}"'


In [ ]:
# Calibrate hooks (throw hooks for 2.5 seconds after running)
!bash -lc 'source /tmp/boxbunny_env.sh && ros2 service call /calibrate_imu_punch boxbunny_msgs/srv/CalibrateImuPunch "{punch_type: hook, duration_s: 2.5}"'


In [ ]:
# Calibrate uppercuts (throw uppercuts for 2.5 seconds after running)
!bash -lc 'source /tmp/boxbunny_env.sh && ros2 service call /calibrate_imu_punch boxbunny_msgs/srv/CalibrateImuPunch "{punch_type: uppercut, duration_s: 2.5}"'


---
## 4. LLM Coach Tuning
Tune the LLM for trash talk, encouragement, and analysis.


In [ ]:
# Launch LLM Chat GUI for prompt tuning
!bash -lc 'source /tmp/boxbunny_env.sh && ros2 run boxbunny_llm llm_chat_gui'


In [ ]:
# Test LLM directly via service call


---
## 5. Drill Configuration
Test individual drill modes.


In [ ]:
# Start a shadow sparring drill (1-1-2 combo)
!bash -lc 'source /tmp/boxbunny_env.sh && ros2 service call /start_drill boxbunny_msgs/srv/StartDrill "{drill_type: shadow_sparring, drill_name: \"1-1-2 Combo\", repetitions: 1}"'


In [ ]:
# Monitor drill progress
!bash -lc 'source /tmp/boxbunny_env.sh && ros2 topic echo -n 5 /drill_progress'


In [ ]:
# Monitor action predictions
!bash -lc 'source /tmp/boxbunny_env.sh && ros2 topic echo -n 10 /action_prediction'


---
## 6. Full System Integration
Launch and test the complete system.


In [ ]:
# List all active topics
!bash -lc 'source /tmp/boxbunny_env.sh && ros2 topic list'


In [ ]:
# Check node status
!bash -lc 'source /tmp/boxbunny_env.sh && ros2 node list'


In [ ]:
# Launch full system (GUI + IMU + LLM + Action Prediction)
!bash -lc 'source /tmp/boxbunny_env.sh && ros2 launch boxbunny_bringup boxbunny_system.launch.py enable_imu:=true enable_llm:=true enable_gui:=true'
